# Cleaning unique user data

In [1]:
# importing necessary packages
import pandas as pd
import requests
from dotenv import load_dotenv
load_dotenv()
import os
import json
import flickrapi
import time

In [2]:
# read csv into df
df_user_data = pd.read_csv('data/unique_owner.csv')

/var/folders/n9/f309cdzj1vg84wrrq1h5nskm0000gn/T/ipykernel_53278/63229535.py:2: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_user_data = pd.read_csv('data/unique_owner.csv')


In [3]:
# check df
df_user_data.head(10)

,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,stat,person_id,person_nsid,...,person_has_free_educational_resources,person_timezone_label,person_timezone_offset,person_timezone_timezone_id,person_timezone_timezone,person_pro_badge,person_expire,person_mbox_sha1sum__content,code,message
0,0,0.0,0.0,0.0,0.0,0.0,0.0,ok,197843899@N04,197843899@N04,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,1.0,1.0,1.0,1.0,1.0,ok,26745338@N06,26745338@N06,...,0.0,Brasilia,-03:00,America/Sao_Paulo,20.0,NaN,NaN,NaN,NaN,NaN
2,2,2.0,2.0,2.0,2.0,2.0,2.0,ok,31331368@N00,31331368@N00,...,0.0,"Canberra, Melbourne, Sydney",+10:00,Australia/Canberra,68.0,standard,0.000000e+00,NaN,NaN,NaN
3,3,3.0,3.0,3.0,3.0,3.0,3.0,ok,122687277@N03,122687277@N03,...,0.0,NaN,NaN,NaN,NaN,standard,0.000000e+00,NaN,NaN,NaN
4,4,4.0,4.0,4.0,4.0,4.0,4.0,ok,196275842@N08,196275842@N08,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5.0,5.0,5.0,5.0,5.0,5.0,ok,44842144@N03,44842144@N03,...,0.0,NaN,NaN,NaN,NaN,standard,0.000000e+00,NaN,NaN,NaN
6,6,6.0,6.0,6.0,6.0,6.0,6.0,ok,45008553@N04,45008553@N04,...,0.0,"Bangkok, Hanoi, Jakarta",+07:00,Asia/Bangkok,55.0,NaN,NaN,NaN,NaN,NaN
7,7,7.0,7.0,7.0,7.0,7.0,7.0,ok,158117692@N02,158117692@N02,...,0.0,NaN,NaN,NaN,NaN,standard,1.614069e+09,NaN,NaN,NaN
8,8,8.0,8.0,8.0,8.0,8.0,8.0,ok,118071797@N03,118071797@N03,...,0.0,NaN,NaN,NaN,NaN,standard,0.000000e+00,NaN,NaN,NaN
9,9,9.0,9.0,9.0,9.0,9.0,9.0,ok,169491036@N06,169491036@N06,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Drop unnecessary columns

In [4]:
# check value count for is deleted
print(df_user_data.person_is_deleted.value_counts())
print(df_user_data.person_is_deleted.isna().sum())
# because there is only 1 value (False) and 24 NaN I decided to drop the column

0.0    38777
Name: person_is_deleted, dtype: int64
24


In [5]:
# list of columns to drop
columns_to_drop = ['Unnamed: 0.6','Unnamed: 0.5','Unnamed: 0.4', 'Unnamed: 0.3',
       'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'stat','person_nsid','person_is_deleted',
       'person_iconserver','person_iconfarm','person_has_stats',
       'person_description__content',
       'person_photosurl__content', 'person_profileurl__content',
       'person_mobileurl__content','person_has_adfree', 'person_has_free_standard_shipping',
       'person_has_free_educational_resources','person_pro_badge','person_mbox_sha1sum__content','message','person_expire', 'code', 'person_timezone_timezone']

In [6]:
# check if its the right number
len(columns_to_drop)

25

In [7]:
# drop 25 columns
df_user_data_cleaned = df_user_data.drop(columns=columns_to_drop)

In [8]:
# check columns
df_user_data_cleaned.columns

Index(['person_id', 'person_ispro', 'person_path_alias',
       'person_username__content', 'person_realname__content',
       'person_location__content', 'person_photos_firstdatetaken__content',
       'person_photos_firstdate__content', 'person_photos_count__content',
       'person_timezone_label', 'person_timezone_offset',
       'person_timezone_timezone_id', 'person_timezone_timezone'],
      dtype='object')

## Alter column names

In [9]:
# replace 'person' with user and delete '_content'
df_user_data_cleaned.columns = df_user_data_cleaned.columns.str.replace('person','user').str.replace('__content','')


In [10]:
df_user_data_cleaned.columns

Index(['user_id', 'user_ispro', 'user_path_alias', 'user_username',
       'user_realname', 'user_location', 'user_photos_firstdatetaken',
       'user_photos_firstdate', 'user_photos_count', 'user_timezone_label',
       'user_timezone_offset', 'user_timezone_timezone_id',
       'user_timezone_timezone'],
      dtype='object')

## delete NaN from user_location, bc it's our most important column

In [11]:
# clear NaN values from user_locations
df_user_data_cleaned = df_user_data_cleaned.dropna(subset='user_location')
df_user_data_cleaned = df_user_data_cleaned.reset_index(drop=True)

In [12]:
# check how much is still there
df_user_data_cleaned.shape

(14469, 13)

In [13]:
len(df_user_data)

38801

In [14]:
# check percentage of leftover data
x = (len(df_user_data_cleaned)/len(df_user_data))*100
print(x)

37.29027602381382


## alter dtypes

In [15]:
# check dtypes
df_user_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14469 entries, 0 to 14468
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     14469 non-null  object 
 1   user_ispro                  14469 non-null  float64
 2   user_path_alias             9890 non-null   object 
 3   user_username               14469 non-null  object 
 4   user_realname               12591 non-null  object 
 5   user_location               14469 non-null  object 
 6   user_photos_firstdatetaken  14468 non-null  object 
 7   user_photos_firstdate       14468 non-null  float64
 8   user_photos_count           14469 non-null  float64
 9   user_timezone_label         13283 non-null  object 
 10  user_timezone_offset        13283 non-null  object 
 11  user_timezone_timezone_id   13283 non-null  object 
 12  user_timezone_timezone      13283 non-null  float64
dtypes: float64(4), object(9)
memory

In [16]:
# user_id to string
df_user_data_cleaned.user_id=df_user_data_cleaned.user_id.astype('string')

In [17]:
df_user_data_cleaned.user_id

0         26745338@N06
1         31331368@N00
2        122687277@N03
3         45008553@N04
4        158117692@N02
             ...      
14464    131418876@N03
14465     42905557@N00
14466     16939187@N06
14467     97454045@N07
14468     35939355@N04
Name: user_id, Length: 14469, dtype: string

In [18]:
# user_ispro to bool
df_user_data_cleaned.user_ispro=df_user_data_cleaned.user_ispro.astype('bool')

In [19]:
# user_location','user_realname','user_username','user_path_alias to string
df_user_data_cleaned[['user_location','user_realname','user_username','user_path_alias']] = df_user_data_cleaned[['user_location','user_realname','user_username','user_path_alias']].astype('string')

In [20]:
# user_photos_count to int
df_user_data_cleaned.user_photos_count=df_user_data_cleaned.user_photos_count.astype('int')

In [21]:
df_user_data_cleaned.user_photos_firstdate

0        1.212696e+09
1        1.099811e+09
2        1.397393e+09
3        1.259290e+09
4        1.515479e+09
             ...     
14464    1.424362e+09
14465    1.134207e+09
14466    1.197543e+09
14467    1.371249e+09
14468    1.251922e+09
Name: user_photos_firstdate, Length: 14469, dtype: float64

In [22]:
# user_firstdate unix to datetime
df_user_data_cleaned.user_photos_firstdate = pd.to_datetime(df_user_data_cleaned.user_photos_firstdate,unit='s',errors='coerce')

In [27]:
df_user_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14469 entries, 0 to 14468
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   user_id                     14469 non-null  string        
 1   user_ispro                  14469 non-null  bool          
 2   user_path_alias             9890 non-null   string        
 3   user_username               14469 non-null  string        
 4   user_realname               12591 non-null  string        
 5   user_location               14469 non-null  string        
 6   user_photos_firstdatetaken  14468 non-null  string        
 7   user_photos_firstdate       14468 non-null  datetime64[ns]
 8   user_photos_count           14469 non-null  int64         
 9   user_timezone_label         13283 non-null  object        
 10  user_timezone_offset        13283 non-null  object        
 11  user_timezone_timezone_id   13283 non-null  object    

In [25]:
# check type
type(df_user_data_cleaned.user_photos_firstdatetaken)

pandas.core.series.Series

In [26]:
# alter to string
df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.astype('string')

In [ ]:
df_user_data_cleaned.user_photos_firstdatetaken[9]

In [ ]:
len(df_user_data_cleaned.user_photos_firstdatetaken[37])

In [ ]:
# for i, date_taken in enumerate(df_user_data_cleaned['user_photos_firstdatetaken']):
#     length = len(date_taken)
#     if length != 19:
#         print(f"Row {i + 1}: Length of 'user_photos_firstdatetaken': {length}")
    


In [ ]:
# df_user_data_cleaned = df_user_data_cleaned.dropna(subset='user_photos_firstdatetaken')
# creepy_dates=[]
# for i in df_user_data_cleaned.user_photos_firstdatetaken:
#     if len(i) is not len(df_user_data_cleaned.user_photos_firstdatetaken[1]) :
#         creepy_dates.append(i) 
#         print('jo')
#     else:
#         print('buh')

In [ ]:
# creepy_dates = []
# for i in df_user_data_cleaned.user_photos_firstdatetaken:
#     if len(i) == 19:
#         print('jo')
#     else:
#         creepy_dates.append(i)
#         print('buh')

In [ ]:
#len(creepy_dates)

In [ ]:
# creepy_dates = df_user_data_cleaned['user_photos_firstdatetaken'].apply(lambda x: len(x) != '19')

# for i, is_creepy in enumerate(creepy_dates):
#     if is_creepy:
#         print(f'Element {i} is creepy')
#     else:
#         print(f'Element {i} is not creepy')


In [ ]:

# for i, row in df_user_data_cleaned.iterrows():
#     try:
#         df_user_data_cleaned.user_photos_firstdatetaken=pd.to_datetime(row.get('user_photos_firstdatetaken'), format='%Y-%m-%d %H:%M:%S')
#     except ValueError:
#         print(str(i), row.get('user_photos_firstdatetaken'))

In [ ]:
# df_user_data_cleaned = df_user_data_cleaned.dropna(subset='user_photos_firstdatetaken')
# counter= 0
# for i,row in enumerate(df_user_data_cleaned.user_photos_firstdatetaken):
#     if row == '0000-00-00 00:00:00':
#         counter+= 1
#         print(i,row,counter)

In [28]:
df_user_data_cleaned.user_photos_firstdatetaken[9:15]

9     1840-01-01 00:00:00
10    2000-01-01 00:00:00
11    0000-00-00 00:00:00
12    0000-00-00 00:00:00
13    1980-01-01 00:00:07
14    1985-03-15 00:00:00
Name: user_photos_firstdatetaken, dtype: string

In [29]:
df_user_data_cleaned.user_photos_firstdatetaken = pd.to_datetime(df_user_data_cleaned.user_photos_firstdatetaken, dayfirst=True,format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [31]:
# because coerce replaces error values with NaT 
# firstdatetaken is mysql datetime that starts with 1000-00-00 00:00:00 we lost 787 values
df_user_data_cleaned.user_photos_firstdatetaken.count()

13681

In [ ]:
#df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.fillna('0000-00-00 00:00:00')

In [ ]:
#df_user_data_cleaned.user_location

In [ ]:
#df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.astype('str')

In [ ]:
#failed because of '0000-00-00 00:00:00'
#pd.to_datetime(df_user_data_cleaned.user_photos_firstdatetaken, dayfirst=True)

In [ ]:
#df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.str.replace('0000-00-00 00:00:00','1800-01-01 00:00:00')

In [ ]:
#df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.str.replace('0-01-01 00:00:00','1800-01-01 00:00:00')

In [ ]:
#df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.str.replace('1841800-01-01 00:00:00','1800-01-01 00:00:00')

In [ ]:
#df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.str.replace('2001800-01-01 00:00:00','1800-01-01 00:00:00')

In [ ]:
#df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.str.replace('1801800-01-01 00:00:00','1800-01-01 00:00:00')

In [ ]:
#df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.str.replace('1891800-01-01 00:00:00','1800-01-01 00:00:00')

In [33]:
df_user_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14469 entries, 0 to 14468
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   user_id                     14469 non-null  string        
 1   user_ispro                  14469 non-null  bool          
 2   user_path_alias             9890 non-null   string        
 3   user_username               14469 non-null  string        
 4   user_realname               12591 non-null  string        
 5   user_location               14469 non-null  string        
 6   user_photos_firstdatetaken  13681 non-null  datetime64[ns]
 7   user_photos_firstdate       14468 non-null  datetime64[ns]
 8   user_photos_count           14469 non-null  int64         
 9   user_timezone_label         13283 non-null  object        
 10  user_timezone_offset        13283 non-null  object        
 11  user_timezone_timezone_id   13283 non-null  object    

In [ ]:
df_user_data_cleaned.duplicated().value_counts()

In [ ]:
# check format of user_location
df_user_data_cleaned.user_location

In [ ]:
# try to split user_location
df_split = df_user_data_cleaned.user_location.str.split(', ', expand=True)
df_split.sample(2)

In [ ]:
#catch some outliers to identify the split problem
df_user_data_cleaned.iloc[8048:8053]

In [ ]:
# split without expand=true
df_split = df_user_data_cleaned.user_location.str.split(', ')

In [ ]:
# check whats going on with 21 columns
df_split.sample(15)

In [ ]:
# check type of df_split
type(df_split)

In [ ]:
# change dt to list
df_split.to_list()

In [ ]:
one_list=[]
for x in df_split:
    if len(x)<=1:
        one_list.append(x)

In [ ]:
df_split.to_list()

In [ ]:
three_list=[]
for x in df_split:
    if len(x)>=3:
        three_list.append(x)

In [ ]:
three_list

In [ ]:
two_list=[]
for x in df_split:
    if len(x)>=2:
        two_list.append(x)

In [ ]:
two_list

In [ ]:
four_list=[]
for x in df_split:
    if len(x)>=4:
        four_list.append(x)

In [ ]:
twone_list=[]
for x in df_split:
    if len(x)>=21:
        twone_list.append(x)

In [ ]:
twone_list

In [ ]:
print(len(one_list))
print(len(two_list))
print(len(three_list))
print(len(four_list))

In [ ]:
one_list

In [ ]:
df_user_data_cleaned['user_city']

In [ ]:
# split location into country and city
df_user_data_cleaned[['user_city','user_country']] = df_user_data_cleaned.user_location.str.split(n=1, expand=True)[1]

In [ ]:
df_user_data_cleaned[['user_city','user_country']] = df_user_data_cleaned.apply(lambda row: row['user_location'].split(), axis=1)